# packages 

In [1]:
import numpy as np
import random
import pandas as pd

## Transition matrix

In [2]:
def get_security_transition_matrix():
    # The transition matrix is the same regardless if the game is cycled or not
    # /!\ index different from square number (need to do -1)
    n_squares = 15 
    T_m = np.zeros((n_squares, n_squares))
    #i = index from ==> on ne commence jamais à cet index car on gagné à ce moment. La proba =1. C'est mis après 
    for i in range(n_squares-1): 
        T_m[i][i] = 1/2
        if(i==2):
            T_m[i][i+1] = 1/4 #if takes slow line
            T_m[i][10] = 1/4 #if takes fast line
        elif(i==9):
            T_m[i][14] = 1/2            
        else:
            T_m[i][i+1] = 1/2
    T_m[14][14] = 1
    return T_m

def get_normal_transition_matrix(layout, cycle:bool):
    # The transition matrix is the same regardless tif the game is cycled or not
    # If the player just passes through square 3 without stopping, 
    # he continues to square 4 or beyond (5, 6, etc), as if the other path does not exist
    n_squares = 15 
    T_m = np.zeros((n_squares, n_squares))
    for i in range(n_squares-1): # o
        T_m[i][i] = 1/3
        if(i==2):
            # #if takes slow line
            T_m[i][i+1] = 1/6 
            T_m[i][i+2] = 1/6
            #if takes fast line
            T_m[i][10] = 1/6
            T_m[i][11] = 1/6

        elif(i==8):
            T_m[i][i+1] = 1/3
            T_m[i][14] = 1/3

        elif(i==9 or i==13):
            if(cycle): 
                T_m[i][14] = 1/3
                T_m[i][0] = 1/3
            else:
                T_m[i][14] = 2/3

        else:
            T_m[i][i+1] = 1/3
            T_m[i][i+2] = 1/3
    T_m[n_squares-1][n_squares-1] = 1

    
    # Il faut encore prendre les pièges et bonuses en compte 
    #il faut reparcourir Chaque élément de la T_m
    for i in range(n_squares-1): # on ne veut pas changer le point de départ ni le point d'arrivé car il n'y a ni piège 
        for j in range(n_squares-1):
                # une chance sur 2 de trigger le piège ou bonus
                if(layout[j]==1 and j!=0): # type 1: go back at 1st square
                    T_m[i][0] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                    T_m[i][j] /=2 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                elif (layout[j]==2 and j!=0): #type 2: go back 3 squares
                    #il y a qlq cas spéciaux
                    if (j>=10 and j <=12): # if j ==11 12 or 13 --> go to 1,2 or 3 with proba 1/2
                        #diff = 10
                        T_m[i][j-10] += T_m[i][j]/2
                        T_m[i][j] /=2 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                        
                    # si trap type 2 à la case 2 ou 3, on peut pas aller moins loin que le start 
                    elif(j<3):
                        T_m[i][0] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                        T_m[i][j] /=2 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                        
                    else: # cas "commun"
                        T_m[i][j-3] += T_m[i][j]/2
                        T_m[i][j] /=2 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                
    return T_m

def get_risky_transition_matrix(layout, cycle:bool):
    
    n_squares = 15
    T_m = np.zeros((n_squares, n_squares))

    for i in range(n_squares-1): # on prend le dernier square après la boucle, d'où le -1

        T_m[i][i] = 1/4
        if(i==2):
            # #if takes slow line
            T_m[i][i+1] = 1/8 
            T_m[i][i+2] = 1/8
            T_m[i][i+3] = 1/8
            #if takes fast line
            T_m[i][10] = 1/8
            T_m[i][11] = 1/8
            T_m[i][12] = 1/8
        
        elif(i==7):
            T_m[i][i+1] = 1/4
            T_m[i][i+2] = 1/4
            T_m[i][14] = 1/4 #(du 10 au 15)

        elif(i==8 or i==12):
            T_m[i][9] = 1/4
            if cycle :
                T_m[i][14] = 1/4
                T_m[i][0] = 1/4
            else : 
                T_m[i][14] = 1/2
        
        elif(i==9 or i == 13):
            if cycle :
                T_m[i][14] = 1/4
                T_m[i][0] = 1/4
                T_m[i][1] = 1/4
            else :
                T_m[i][14] = 3/4
        
        else : 
            T_m[i][i+1] = 1/4
            T_m[i][i+2] = 1/4
            T_m[i][i+3] = 1/4
    
    T_m[n_squares-1][n_squares-1] = 1
    
    
    # LES PIEGES :
    for i in range(n_squares-1): # on ne veut pas changer le point de départ ni le point d'arrivé car il n'y a ni piège 
        for j in range(n_squares-1):
            # Le trap est d'office triggered si c'en est un
            if(layout[j]==1 and j!=0): # type 1: go back at 1st square # J'ai ajouté and j!=! pcq ca rajoutait la condition pour soit même.
                T_m[i][0] += T_m[i][j]
                T_m[i][j] = 0 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément

            elif(layout[j]== 2 and j!=0): # -3 squares
                if (j>=10 and j <=12):
                        #diff = 10
                    T_m[i][j-10] += T_m[i][j]
                    T_m[i][j] = 0 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                # si trap type 2 à la case 2 ou 3, on peut pas aller moins loin que le start 
                elif(j<3):
                    T_m[i][0] += T_m[i][j] # il faut transférer une demi proba vers le square 1
                    T_m[i][j] = 0 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
                else: # cas "commun"
                    T_m[i][j-3] += T_m[i][j]
                    T_m[i][j] = 0 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
            

                    

    return T_m

### Check if sum of probabilites equal to 1 for each row.

In [4]:
L = [random.randrange(0, 5, 1) for i in range(15)]

print(L)
print(get_normal_transition_matrix(L, True))
print("---------------------------------------")
print(get_security_transition_matrix())

my_array = get_normal_transition_matrix(L, False)
np.sum(my_array,axis=1).tolist()

[2, 3, 2, 3, 1, 0, 0, 2, 2, 0, 2, 2, 4, 4, 2]
[[0.5        0.33333333 0.16666667 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.16666667 0.33333333 0.16666667 0.33333333 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.33333333 0.08333333 0.16666667 0.16666667 0.08333333 0.
  0.         0.         0.         0.         0.08333333 0.08333333
  0.         0.         0.        ]
 [0.16666667 0.         0.         0.33333333 0.16666667 0.33333333
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.16666667 0.         0.         0.         0.16666667 0.33333333
  0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.16666667 0.33333333
  0.33333333 0.16666667 0.         0.         0.         0.
  0.         0.       

[0.9999999999999999,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

## Game Simulation

In [5]:
class SimulateAGame():
    def __init__(self, policy, layout, circle):
        self.layout =layout
        self.circle = circle
        self.policy = policy
        self.cost = 0

    def get_cost_game(self):
        """
        policy says which dice to take via int number: 
        - 1 : security
        - 2 : normal 
        - 3 : risky
        """
        self.cost = 0
        return self.get_cost_from(0)
    
    def get_cost_from(self, index_from):
        """ 
        """
        #assert(len(policy) == len(self.layout))
        current_index = index_from 
        while(current_index != 14):
            dice = self.policy[current_index] #{1,2,3}
            if(dice not in [1,2,3]): raise ValueError('sorry zobi')
            current_index = self.play(current_index, dice)
            self.cost+=1    
        return self.cost

    def play(self, current_index, dice_type):
        """return resulting index"""        
        if( dice_type == 1):  # no traps nor bonuses
            if(current_index == 2): return np.random.choice([2,3,10], p = [0.5, 0.25, 0.25] )
            elif (current_index == 9): return np.random.choice([9,14] )
            else: return np.random.choice([current_index, current_index+1])
        elif(dice_type ==2): # normal dice: 1/2 to trigger trap/bonus
            is_triggered = np.random.choice([True,False]) #50% chance to trigger trap/bonus

            if(current_index==2): next_index = np.random.choice([current_index,3,4,10,11], p = [1/3, 1/6, 1/6, 1/6, 1/6])
            elif(current_index==8): next_index = np.random.choice([current_index,9,14])
            elif(current_index==9 or current_index == 13):
                if(self.circle): 
                    next_index = np.random.choice([current_index,14,0])
                else: next_index = np.random.choice([current_index,14], p = [1/3,2/3])

            # Ne faut il pas ajouté pour tous les autres cas ?
            else : next_index = np.random.choice([current_index, current_index+1, current_index+2])
                
            if (is_triggered and next_index!=14): return self.activateTrapOrBonus(next_index)
            else: return next_index

        else: # type =3, i.e. risky
            if(current_index==2):
                next_index = np.random.choice([current_index,3,4,5, 10,11,12], p = [1/4, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8])
            elif(current_index==7):
                next_index = np.random.choice([current_index,8,9,14])
            elif(current_index==8):
                if(self.circle): next_index = np.random.choice([current_index,9,14,0])
                else: next_index = np.random.choice([current_index,9,14], p = [1/4, 1/4, 2/4])
            elif(current_index==9):
                if(self.circle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            elif(current_index==12):
                if(self.circle): next_index = np.random.choice([current_index,13,14,0])
                else: next_index = np.random.choice([current_index,13,14], p = [1/4, 1/4, 2/4])
            elif(current_index==13):
                if(self.circle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            
            else: next_index = np.random.choice([current_index, current_index+1 , current_index+2, current_index+3])

            return self.activateTrapOrBonus(next_index)

    def activateTrapOrBonus(self,index):
        if (self.layout[index] == 1): return self.activateType1Trap(index)
        elif (self.layout[index] == 2): return self.activateType2Trap(index)
        elif (self.layout[index] == 3): return self.activateType3Trap(index)
        elif (self.layout[index] == 4): return self.activateBonus(index)
        else: return index # ordinary square (0)

    def activateType1Trap(self, index):
        if (index!=14):
            return 0
        else : return index


    def activateType2Trap(self, index):
        if (index!=14):
            if(index<3 or index == 10): 
                return 0 # or index = 10 je crois
            elif(index == 11 or index == 12): 
                return (index-10) # Entre 11 et 12 et non entre 10 et 12
            else: 
                return (index-3)
        else : return index

    def activateType3Trap(self, index):
        if (index!=14):
            self.cost += 1
        return index

    def activateBonus(self, index):
        if (index!=14):
            self.cost -= 1
        return index
    

    def get_policy_history(self, policy= None):
        if policy is None: policy = self.policy # si on ne met rien dedans, ça met la policy de base. Mais on peut voir ce que ça fait selon une politique donnée 
        history = pd.DataFrame(columns= ["Nb_of_throws", "action", "current_index", "layout_info" ])
        current_index = 0 
        current_cost = 0
        while(current_index != 14):
            dice = policy[current_index] #{1,2,3}
            if(dice not in [1,2,3]): raise ValueError('sorry zobi')
            # Adding information to history:
            history.loc[len(history.index)] = [current_cost, dice, current_index, self.layout[current_index] ]
            current_index = self.play(current_index, dice)
            #update number of throws
            current_cost+=1    
            
        return history

## Class Markov decision

In [6]:
class MarkovDecision():

    def __init__(self, layout:  np.ndarray, cycle:bool):
        self.layout = layout
        self.cycle = cycle
        self.cost = 0
        self.security_transition_matrix = get_security_transition_matrix()
        self.normal_transition_matrix = get_normal_transition_matrix(layout,cycle)
        self.get_risky_transition_matrix = get_risky_transition_matrix(layout,cycle)
    
    def play(self, current_index, dice_type):
        """return resulting index"""        
        if( dice_type == 1):  # no traps nor bonuses
            if(current_index == 2): return np.random.choice([2,3,10], p = [0.5, 0.25, 0.25] )
            elif (current_index == 9): return np.random.choice([9,14] )
            else: return np.random.choice([current_index, current_index+1])
        elif(dice_type ==2): # normal dice: 1/2 to trigger trap/bonus
            is_triggered = np.random.choice([True,False]) #50% chance to trigger trap/bonus

            if(current_index==2): next_index = np.random.choice([current_index,3,4,10,11], p = [1/3, 1/6, 1/6, 1/6, 1/6])
            elif(current_index==8): next_index = np.random.choice([current_index,9,14])
            elif(current_index==9 or current_index == 13):
                if(self.cycle): 
                    next_index = np.random.choice([current_index,14,0])
                else: next_index = np.random.choice([current_index,14], p = [1/3,2/3])

            # Ne faut il pas ajouté pour tous les autres cas ?
            else : next_index = np.random.choice([current_index, current_index+1, current_index+2])
                
            if (is_triggered and next_index!=14): return self.activateTrapOrBonus(next_index)
            else: return next_index

        else: # type =3, i.e. risky
            if(current_index==2):
                next_index = np.random.choice([current_index,3,4,5, 10,11,12], p = [1/4, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8])
            elif(current_index==7):
                next_index = np.random.choice([current_index,8,9,14])
            elif(current_index==8):
                if(self.cycle): next_index = np.random.choice([current_index,9,14,0])
                else: next_index = np.random.choice([current_index,9,14], p = [1/4, 1/4, 2/4])
            elif(current_index==9):
                if(self.cycle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            elif(current_index==12):
                if(self.cycle): next_index = np.random.choice([current_index,13,14,0])
                else: next_index = np.random.choice([current_index,13,14], p = [1/4, 1/4, 2/4])
            elif(current_index==13):
                if(self.cycle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            
            else: next_index = np.random.choice([current_index, current_index+1 , current_index+2, current_index+3])

            return self.activateTrapOrBonus(next_index)

    def activateTrapOrBonus(self,index):
        if (self.layout[index] == 1): return self.activateType1Trap(index)
        elif (self.layout[index] == 2): return self.activateType2Trap(index)
        elif (self.layout[index] == 3): return self.activateType3Trap(index)
        elif (self.layout[index] == 4): return self.activateBonus(index)
        else: return index # ordinary square (0)

    def activateType1Trap(self, index):
        if (index!=14):
            return 0
        else : return index


    def activateType2Trap(self, index):
        if (index!=14):
            if(index<3 or index == 10): 
                return 0 # or index = 10 je crois
            elif(index == 11 or index == 12): 
                return (index-10) # Entre 11 et 12 et non entre 10 et 12
            else: 
                return (index-3)
        else : return index

    def activateType3Trap(self, index):
        if (index!=14):
            self.cost += 1
        return index

    def activateBonus(self, index):
        if (index!=14):
            self.cost -= 1
        return index

    def compute_expected_cost(self, index: int, V_hat: np.array):
        if (index in list(range(0,15))):
            secur = np.dot(get_security_transition_matrix()[index], V_hat)
            normal = np.dot(get_normal_transition_matrix(layout=self.layout, cycle=self.cycle)[index], V_hat)
            risky = np.dot(get_risky_transition_matrix(layout=self.layout, cycle=self.cycle)[index], V_hat)

            return(secur, normal, risky)
            # if we want to directly know which dice to choose 
            """
            print(secur,' || ',  normal)

            result = np.array([secur, normal])
            min = np.min(result)
            for i,el in enumerate(result):
                if el == min:
                    dice = i
            return dice+1"""

    
    def get_policy_history(self, policy=None):
        if policy is None: policy = self.policy # si on ne met rien dedans, ça met la policy de base. Mais on peut voir ce que ça fait selon une politique donnée 
        history = pd.DataFrame(columns= ["Nb_of_throws", "action", "current_index", "layout_info" ])
        current_index = 0 
        current_cost = 0
        while(current_index != 14):
            dice = policy[current_index] #{1,2,3}
            if(dice not in [1,2,3]): raise ValueError('sorry zobi')
            # Adding information to history:
            history.loc[len(history.index)] = [current_cost, dice, current_index, self.layout[current_index] ]
            current_index = self.play(current_index, dice)
            #update number of throws
            current_cost+=1    
            
        return history

In [7]:
L = [0,2,3,0,0,3,2,0,2,3,0,0,3,0,0]
secu_policy = [1,2,2,1,1,1,1,1,1,1,1,1,1,3,1]
aGame = MarkovDecision(layout=L, cycle=True)
aGame.compute_expected_cost(2, L)

(1.5, 1.0, 1.5)

In [8]:
def markovDecision(layout, cycle):
    # déclaration/ instanciations
    mdp = MarkovDecision(layout, cycle)
    V_hat = np.ones(15)

    index = 0
    diff_v_hat = 100
    count = 0
    while(count < 100000):
        while(index != 14):
            # to know wich dice I choose
            expected_cost = mdp.compute_expected_cost(index, V_hat)
            min_expected_cost = np.min(expected_cost)
            for i,el in enumerate(expected_cost):
                if el == min_expected_cost:
                    dice = i+1

            # where I am landing to 
            next_index = mdp.play(index, dice)
            # update the V_k 
            V_hat[index] += min_expected_cost
            # update index 
            index = next_index
            #print(index)
        count+=1
    print (count)
    return V_hat

markovDecision(layout=[0]*15, cycle=True) 

100000


array([2. , 1. , 1. , 2. , 2. , 2. , 1. , 1. , 2. , 3.5, 1. , 1. , 1. ,
       1. , 1. ])

In [9]:
class Board():
    # here we place class attribute (static members)
    # init 
    def __init__(self, layout:  np.ndarray, cycle=False):
        self.layout = layout
        self.cycle = cycle
        # self.squares = np.ones(15) # matrice 15 x 3 avec les actions et expected costs associés. 
        self.position = 0 #position actuelle du joueur
    
    # regular step
    def step(self, size):
        self.position += size

    # reset (in case of circular)
    def reset_position(self):
        self.position = 0

    def get_position(self):
        return self.position
    
    def print(self):
        pass

In [10]:
class Dice():
    def __init__(self, type:int):
        self.last_toss = None

        if(type == 1):
            self.steps = np.array([0,1])
        elif (type == 2):
            self.steps = np.array([0,1,2])
        elif (type== 3):
            self.steps = np.array([0,1,2,3])
        else: raise ValueError(' /!\  Dice type not existing in current settings')
        

    def roll(self):
        """ 
        draw uniformly a step from the dice
        """
        self.last_toss =  int(np.random.choice(self.steps, size=1))
        return self.last_toss
    
    def get_current_toss(self):
        if(self.last_toss==None): raise ValueError(' /!\ Dice should have been tossed at least once')
        return self.last_toss

In [11]:
def get_proba_next_move_position(v_k, T_m):
    # v_k = current_state proba vector
    # return next state proba vector
    return np.dot(v_k, T_m)

In [12]:
v_0 = np.zeros(16)
v_0[1] = 1
security_transition_matrix = get_security_transition_matrix()
np.dot(v_0, security_transition_matrix) # représente la proba de la prochaine position

ValueError: shapes (16,) and (15,15) not aligned: 16 (dim 0) != 15 (dim 0)

## Example of Value iteration Algorithm

In [30]:
import numpy as np
import random



'''==================================================
Initial set up
=================================================='''

L = [random.randrange(0, 5, 1) for i in range(15)]
L[0] = L[-1] = 0

#Hyperparameters
SMALL_ENOUGH = 0.005
GAMMA = 0.9 # gamma n'est pas utile    
NOISE = 0.1 

all_states= list(range(15))

rewards = []
print(L)
for i in range(15) :
    if(L[i] == 4) :
        rewards.append(1)
    else:
        rewards.append(-1)




[0, 3, 4, 2, 2, 2, 3, 1, 3, 1, 3, 3, 3, 2, 0]


[-1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [10]:
import numpy as np

'''==================================================
Initial set up
=================================================='''

#Hyperparameters
SMALL_ENOUGH = 0.005
GAMMA = 0.9         
NOISE = 0.1  

#Define all states
all_states=[]
for i in range(3):
    for j in range(4):
            all_states.append((i,j))

#Define rewards for all states
rewards = {}
for i in all_states:
    if i == (1,2):
        rewards[i] = -1
    elif i == (2,2):
        rewards[i] = -1
    elif i == (2,3):
        rewards[i] = 1
    else:
        rewards[i] = 0

#Dictionnary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {
    (0,0):('D', 'R'), 
    (0,1):('D', 'R', 'L'),    
    (0,2):('D', 'L', 'R'),
    (0,3):('D', 'L'),
    (1,0):('D', 'U', 'R'),
    (1,1):('D', 'R', 'L', 'U'),
    (1,3):('D', 'L', 'U'),
    (2,0):('U', 'R'),
    (2,1):('U', 'L', 'R'),
    }

#Define an initial policy
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions[s])

#Define initial value function 
V={}
for s in all_states:
    if s in actions.keys():
        V[s] = 0
    if s ==(2,2):
        V[s]=-1
    if s == (1,2):
        V[s]=-1
    if s == (2,3):
        V[s]=1

In [12]:
all_states

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [9]:

'''==================================================
Value Iteration
=================================================='''

iteration = 0
while True:
    biggest_change = 0
    for s in all_states:            
        if s in policy:
            
            old_v = V[s]
            new_v = 0
            
            for a in actions[s]:
                if a == 'U':
                    nxt = [s[0]-1, s[1]]
                if a == 'D':
                    nxt = [s[0]+1, s[1]]
                if a == 'L':
                    nxt = [s[0], s[1]-1]
                if a == 'R':
                    nxt = [s[0], s[1]+1]

                #Choose a new random action to do (transition probability)
                random_1=np.random.choice([i for i in actions[s] if i != a])
                if random_1 == 'U':
                    act = [s[0]-1, s[1]]
                if random_1 == 'D':
                    act = [s[0]+1, s[1]]
                if random_1 == 'L':
                    act = [s[0], s[1]-1]
                if random_1 == 'R':
                    act = [s[0], s[1]+1]

                #Calculate the value
                nxt = tuple(nxt)
                act = tuple(act)
                v = rewards[s] + (GAMMA * ((1-NOISE)* V[nxt] + (NOISE * V[act]))) 
                if v > new_v: #Is this the best action so far? If so, keep it
                    new_v = v
                    policy[s] = a

       #Save the best of all actions for the state                                 
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
         
   #See if the loop should stop now         
    if biggest_change < SMALL_ENOUGH:
        break
    iteration += 1